---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2017</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1 style="font-size: 180%;">Version 2</h1>
<h1>Issued Wednesday, 4/13/2017; Due Monday, 4/24/2017</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "John"
    last = "Schutts"
    onyen = "jschutts"
    pid = "720425330"

---
**Problem #1:** Create a suffix array of the [house mouse mitochondrial sequence](http://csbio.unc.edu/mcmillan/Comp555S17/data/MusMusculusMT.fa) used in problem set #3 with the leading '+' replaced by a '&dollar;'. Then find the range suffix-array indices that begin with the "CATTAG" substring.

In [6]:
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

hdr, seq = loadFasta("MusMusculusMT.fa")
genome = seq[0]

def suffixArray(text):
    return sorted(range(len(text)), cmp=lambda i,j: 1 if text[i:] >= text[j:] else -1)

sufArray = suffixArray(genome)

string = 'CATTAG'

def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while(lo < hi):
        middle = (lo+hi)/2
        if text[suffixarray[middle]:]<pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

first = findFirst(string, genome, sufArray)

def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while(lo < hi):
        middle = (lo+hi)/2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

last = findLast(string, genome, sufArray)

print sufArray[first]
print sufArray[last-1]



11566
16003


---
**Problem #2:** Use the suffix array from problem #1 to construct the BWT of the mouse mitochondria. In this BWT find the longest run of repeated bases. Report both the base and how many times it appears.

In [29]:
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

hdr, seq = loadFasta("MusMusculusMT.fa")
genome = seq[0]

def suffixArray(text):
    return sorted(range(len(text)), cmp=lambda i,j: 1 if text[i:] >= text[j:] else -1)

sufArray = suffixArray(genome)

def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)

bwt = BWTfromSuffixArray(genome, sufArray)

length = 0


last = ""
current = 0
maxLen = 0

for i in bwt:
    if i == last:
        current += 1
        if current > maxLen:
            maxLen = current
    else:
        current = 1
        last = i

print(maxLen)
print(last)

#checks for a second base with the same max length
str1 = 'C' * maxLen
str2 = 'T' * maxLen
str3 = 'G' * maxLen

if (str1 in bwt):
    print 'C'

if (str2 in bwt):
    print 'T'

if (str3 in bwt):
    print 'G'
    


10
A
T



---
**Problem #3:** Often it is necessary to search a genomic sequence for a given pattern and its reverse-complement. This can be accomplished using two $O(p)$ queries to the BWT. Suppose instead that we concatenate the given genomic sequence with its reverse-complement with each teminated with a dollar sign and build a BWT of this new string. Will searches in this string find substring in both the given sequence and its reverse complement using just one $O(p)$ query? If not, explain why. If so, decribes any disadvantages of this approach relative to using the original BWR and two queries.

**Problem #4: Programming Problem**

Write a function to compute the LCP array for the implicit suffix array of a given BWT string.

In [2]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

def LCP(bwt, fmindex = None, offset = None):
    if(fmindex is None):
        fmindex, offset = FMIndex(bwt)
        m = len(bwt)
        lcp = [0 for i in xrange(m)]
        prev = recoverSuffix(0, bwt, fmindex, offset)
        for i in xrange(1, m):
            suffix = recoverSuffix(i, bwt, fmindex, offset)
            n = 0
            for a, b in zip(prev, suffix):
                if(a != b):
                    break
                n += 1
            lcp[i] = n
            prev = suffix
        return lcp

**Problem #5:** Use the function that you wrote for Problem #4 to find the largest k-mer that appears 240 or more times in the mouse mitochondria.

In [47]:
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

hdr, seq = loadFasta("MusMusculusMT.fa")
genome = seq[0]

def suffixArray(text):
    return sorted(range(len(text)), cmp=lambda i,j: 1 if text[i:] >= text[j:] else -1)

sufArray = suffixArray(genome)

def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)

bwt = BWTfromSuffixArray(genome, sufArray)

def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

def LCP(bwt, fmindex = None, offset = None):
    if(fmindex is None):
        fmindex, offset = FMIndex(bwt)
        m = len(bwt)
        lcp = [0 for i in xrange(m)]
        prev = recoverSuffix(0, bwt, fmindex, offset)
        for i in xrange(1, m):
            suffix = recoverSuffix(i, bwt, fmindex, offset)
            n = 0
            for a, b in zip(prev, suffix):
                if(a != b):
                    break
                n += 1
            lcp[i] = n
            prev = suffix
        return lcp

lcp = LCP(bwt)

print genome[sufArray[lcp.index(16)]:sufArray[lcp.index(16)]+10]



TATCCTTATT


Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set